<a href="https://colab.research.google.com/github/MajiroZ/for_git_study/blob/master/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##【問題1】スクラッチを振り返る


1. **重みの初期化**
* ランダムな初期値の設定（ゼロでない値）
* 初期化手法（Xavier/Glorot初期化、He初期化など）
* 適切な範囲での重みの設定

2. **エポックのループ処理**
* 訓練データ全体を複数回走査するループ
* 各エポックでの順伝播と逆伝播
* エポックごとの損失値と精度の記録

3. 順伝播の実装
* 入力データの各層への伝播
* 活性化関数の適用
* 出力層までの計算

4. 逆伝播の実装
* 勾配計算
* 損失関数に基づく勾配降下
* パラメータ（重み、バイアス）の更新

5. バッチ処理
* データセットの分割
* ミニバッチ学習の実装
* バッチサイズの決定

6. モデルの評価
* 検証データでの性能測定
* 過学習の検出
* (学習曲線のプロット)

In [1]:
import pandas as pd

# Irisデータセット
try:
    df = pd.read_csv('/Iris.csv')
except FileNotFoundError:
    print("Error: 'Iris.csv' not found. Please make sure the file exists in the current directory.")
    exit()


# 目的変数がIris-versicolorまたはIris-virginicaのデータのみ抽出
df_filtered = df[df['Species'].isin(['Iris-versicolor', 'Iris-virginica'])]

# 結果の表示
df_filtered

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
50,51,7.0,3.2,4.7,1.4,Iris-versicolor
51,52,6.4,3.2,4.5,1.5,Iris-versicolor
52,53,6.9,3.1,4.9,1.5,Iris-versicolor
53,54,5.5,2.3,4.0,1.3,Iris-versicolor
54,55,6.5,2.8,4.6,1.5,Iris-versicolor
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


##【問題2】スクラッチとTensorFlowの対応を考える

1. 重みの初期化
```python
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
    'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
    'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]])
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden1])),
    'b2': tf.Variable(tf.random_normal([n_hidden2])),
    'b3': tf.Variable(tf.random_normal([n_classes]))
}
```
- `tf.random_normal()`を使用してランダムな正規分布に基づく重みを初期化
- `tf.Variable()`で変数として定義し、学習可能なパラメータとしている

2. エポックのループ処理
```python
for epoch in range(num_epochs):
    total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
    total_loss = 0
    for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
        # ミニバッチごとにループ
        sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
        total_loss += loss
```
- `num_epochs`で指定した回数分、学習データ全体を走査
- ミニバッチイテレータを使用してデータを分割
- 各エポックで損失を計算し、累積

3. 順伝播の実装
```python
def example_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output
```
- 各層での行列乗算(`tf.matmul()`)
- 活性化関数の適用（この場合はReLU）
- 最終層の出力計算

4. 逆伝播と最適化
```python
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
```
- 損失関数（シグモイド交差エントロピー）の定義
- Adamオプティマイザを使用
- `minimize()`メソッドで自動的に逆伝播と重み更新を実行

5. モデルの評価
```python
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
```
- 予測と正解ラベルの比較
- 精度（accuracy）の計算

これらの実装で、TensorFlow（この例では1.x系）でディープラーニングモデルの基本的な学習プロセスを実現してる。

##【問題3】3種類すべての目的変数を使用したIrisのモデルを作成

In [24]:
# Irisの3種類の目的変数をすべて使ってTensorFlowで学習・推定するニューラルネットワーク

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
try:
    df = pd.read_csv('/Iris.csv')
except FileNotFoundError:
    print("Error: 'Iris.csv' not found. Please make sure the file exists in the current directory.")
    exit()

# データの前処理
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
X = np.array(X)

# ラベルを数値に変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y = y.astype(np.int64)[:, np.newaxis]


# train, val, testに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


# ミニバッチ処理クラス
class GetMiniBatch:
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]


# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3 #3種類の目的変数

# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("int64", [None, 1]) #目的変数の型をint64に修正

# ワンホットエンコーディング
Y_onehot = tf.one_hot(Y, depth=n_classes)

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# ネットワーク構造の定義
def example_net(x):
    tf.compat.v1.random.set_random_seed(0)
    weights = {
        'w1': tf.Variable(tf.compat.v1.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.compat.v1.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.compat.v1.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.compat.v1.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.compat.v1.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.compat.v1.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

logits = example_net(X)

# 目的関数、最適化手法、評価指標
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_onehot, logits=logits))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(tf.reshape(Y_onehot, [-1, n_classes]), 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.compat.v1.global_variables_initializer()

# 計算グラフの実行
with tf.compat.v1.Session() as sess: # Use tf.compat.v1.Session
    sess.run(init)
    for epoch in range(num_epochs):
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 18.2232, val_loss : 143.0447, acc : 0.292
Epoch 1, loss : 10.1237, val_loss : 72.9604, acc : 0.083
Epoch 2, loss : 4.5031, val_loss : 16.4131, acc : 0.458
Epoch 3, loss : 1.6937, val_loss : 9.7921, acc : 0.375
Epoch 4, loss : 0.6223, val_loss : 1.7191, acc : 0.625
Epoch 5, loss : 0.3016, val_loss : 0.5223, acc : 0.792
Epoch 6, loss : 0.1826, val_loss : 0.3126, acc : 0.875
Epoch 7, loss : 0.1707, val_loss : 0.3292, acc : 0.833
Epoch 8, loss : 0.1429, val_loss : 0.2239, acc : 0.958
Epoch 9, loss : 0.1321, val_loss : 0.2094, acc : 0.958
Epoch 10, loss : 0.1178, val_loss : 0.2746, acc : 0.958
Epoch 11, loss : 0.1078, val_loss : 0.2637, acc : 0.958
Epoch 12, loss : 0.0990, val_loss : 0.3189, acc : 0.958
Epoch 13, loss : 0.0892, val_loss : 0.3346, acc : 0.958
Epoch 14, loss : 0.0796, val_loss : 0.3555, acc : 0.958
Epoch 15, loss : 0.0710, val_loss : 0.3679, acc : 0.958
Epoch 16, loss : 0.0646, val_loss : 0.3736, acc : 0.958
Epoch 17, loss : 0.0591, val_loss : 0.3779, acc : 0.

##【問題4】House Pricesのモデルを作成

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Import MinMaxScaler from sklearn.preprocessing
from sklearn.preprocessing import MinMaxScaler

dataset_path ="/train.csv"
df = pd.read_csv(dataset_path)
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
y = y.astype(int)[:, np.newaxis]
y = np.log(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

tf.compat.v1.disable_eager_execution()

try:
    df = pd.read_csv('/train.csv') # train.csvへのパスを指定
except FileNotFoundError:
    print("Error: 'train.csv' not found. Please make sure the file exists in the current directory.")
    exit()

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]

#欠損値を0で補完
X = X.fillna(0)

X = np.array(X)
y = np.array(y)
y = y.astype(int)[:, np.newaxis]
y = np.log(y)

# train, val, testに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# MinMaxScalerで特徴量をスケーリング
mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

n_input = X_train.shape[1]
n_hidden1 = 50
n_hidden2 = 100
num_epochs = 100
batch_size = 10
learning_rate = 0.001

n_samples = X_train.shape[0]

# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, 1])

# ネットワーク構造の定義
def example_net(x):
    tf.compat.v1.random.set_random_seed(0)
    weights = {
        'w1': tf.Variable(tf.compat.v1.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.compat.v1.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.compat.v1.random_normal([n_hidden2, 1])) # 出力層のユニット数を1に
    }
    biases = {
        'b1': tf.Variable(tf.compat.v1.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.compat.v1.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.compat.v1.random_normal([1])) # 出力層のバイアスも1に
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

logits = example_net(X)

# 目的関数、最適化手法、評価指標
loss_op = tf.reduce_mean(tf.square(logits - Y)) # 平均二乗誤差を使用
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# variableの初期化
init = tf.compat.v1.global_variables_initializer()

# 計算グラフの実行
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(0, len(X_train), batch_size):
          sess.run(train_op, feed_dict={X: X_train[i: i+batch_size], Y: y_train[i: i+batch_size]})
          loss = sess.run(loss_op, feed_dict={X: X_train[i:i+batch_size], Y:y_train[i:i+batch_size]})
          total_loss += loss

        total_loss /= (n_samples / batch_size) # バッチサイズで割る
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, total_loss, val_loss))
    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print("test_loss : {:.3f}".format(test_loss))

Epoch 0, loss : 749.2937, val_loss : 37.2270
Epoch 1, loss : 22.9719, val_loss : 15.2548
Epoch 2, loss : 10.0288, val_loss : 6.2337
Epoch 3, loss : 4.6160, val_loss : 2.6452
Epoch 4, loss : 2.3816, val_loss : 1.2915
Epoch 5, loss : 1.4922, val_loss : 0.8048
Epoch 6, loss : 1.1083, val_loss : 0.5969
Epoch 7, loss : 0.8986, val_loss : 0.4720
Epoch 8, loss : 0.7602, val_loss : 0.3870
Epoch 9, loss : 0.6608, val_loss : 0.3298
Epoch 10, loss : 0.5861, val_loss : 0.2914
Epoch 11, loss : 0.5280, val_loss : 0.2639
Epoch 12, loss : 0.4809, val_loss : 0.2423
Epoch 13, loss : 0.4411, val_loss : 0.2251
Epoch 14, loss : 0.4070, val_loss : 0.2110
Epoch 15, loss : 0.3773, val_loss : 0.1992
Epoch 16, loss : 0.3502, val_loss : 0.1889
Epoch 17, loss : 0.3246, val_loss : 0.1790
Epoch 18, loss : 0.2997, val_loss : 0.1679
Epoch 19, loss : 0.2744, val_loss : 0.1556
Epoch 20, loss : 0.2506, val_loss : 0.1428
Epoch 21, loss : 0.2267, val_loss : 0.1301
Epoch 22, loss : 0.2041, val_loss : 0.1190
Epoch 23, loss 

##【問題5】MNISTのモデルを作成

In [16]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
# Import mnist from tensorflow.keras.datasets
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(float)
X_test = X_test.astype(float)
X_train /= 255
X_test /= 255
y_train = y_train.astype(int)[:, np.newaxis]
y_test = y_test.astype(int)[:, np.newaxis]
enc = OneHotEncoder(handle_unknown='ignore', categories='auto')
y_train_one_hot = enc.fit_transform(y_train[:])
y_test_one_hot = enc.fit_transform(y_test[:])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

In [17]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape and normalize the data
X_train = X_train.reshape(-1, 784).astype(float) / 255
X_test = X_test.reshape(-1, 784).astype(float) / 255

# One-hot encode the labels
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
y_train_one_hot = enc.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = enc.transform(y_test.reshape(-1, 1))

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)

# Define hyperparameters
learning_rate = 0.001
batch_size = 100
num_epochs = 20
n_hidden1 = 50
n_hidden2 = 100
n_input = 784
n_classes = 10

# Define placeholders for input and output
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

# Define the neural network
def example_net(x):
    weights = {
        'w1': tf.Variable(tf.random.normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random.normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random.normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random.normal([n_hidden1])),
        'b2': tf.Variable(tf.random.normal([n_hidden2])),
        'b3': tf.Variable(tf.random.normal([n_classes]))
    }

    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['w1']), biases['b1']))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']))
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    return layer_output

# Build the model
logits = example_net(X)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables
init = tf.compat.v1.global_variables_initializer()

# Start training
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        avg_cost = 0.
        total_batch = int(X_train.shape[0] / batch_size)
        for i in range(total_batch):
            batch_x, batch_y = X_train[i * batch_size:(i + 1) * batch_size], y_train[i * batch_size:(i + 1) * batch_size]
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", '%04d' % (epoch + 1), "cost={:.9f}".format(avg_cost))

    print("Optimization Finished!")
    print("Accuracy:", accuracy.eval({X: X_test, Y: y_test_one_hot}))

Epoch: 0001 cost=85.507478857
Epoch: 0002 cost=19.628206881
Epoch: 0003 cost=12.510167856
Epoch: 0004 cost=9.192040552
Epoch: 0005 cost=7.132923970
Epoch: 0006 cost=5.740982013
Epoch: 0007 cost=4.716303780
Epoch: 0008 cost=3.956058896
Epoch: 0009 cost=3.370950834
Epoch: 0010 cost=2.898652833
Epoch: 0011 cost=2.512472238
Epoch: 0012 cost=2.196542357
Epoch: 0013 cost=1.936289814
Epoch: 0014 cost=1.711242853
Epoch: 0015 cost=1.518095934
Epoch: 0016 cost=1.349413691
Epoch: 0017 cost=1.203014165
Epoch: 0018 cost=1.082966051
Epoch: 0019 cost=0.976389866
Epoch: 0020 cost=0.876413763
Optimization Finished!
Accuracy: 0.8992
